# Classification Networks

In [39]:
from IPython import display
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KernelDensity
from sklearn.metrics import average_precision_score
import torch
from torch import nn, optim
from torch.autograd.variable import Variable
import torch.utils.data as data_utils
import matplotlib
import matplotlib.pyplot as plt

import time
from visdom import Visdom

# Input Data

In [59]:
file_path = "input/creditcard.csv"
df = pd.read_csv(file_path)

x = df.iloc[:, :-1].values
y = df.iloc[:,-1:].values
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.8, random_state = 21, shuffle = True, stratify = y)

# Store Num Features
n_features = x.shape[1]

# Center Mean and Unit Variance
x_train = preprocessing.scale(x_train, axis = 0)
x_test = preprocessing.scale(x_test, axis = 0)

# To Tensor
x_train = torch.from_numpy(x_train).float()
x_test = Variable(torch.from_numpy(x_test).float())
y_train = torch.from_numpy(y_train).double()
y_test = torch.from_numpy(y_test).double()
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)
dtype = torch.float32

# Creates Data Loader
ds_train = data_utils.TensorDataset(x_train, y_train)
data_loader = data_utils.DataLoader(ds_train, batch_size=100, shuffle=True)

using device: cuda


# Visualization

In [23]:
# Visdom Stuff
class VisdomController():
    def __init__(self):
        self.vis = Visdom()
        self.plots = {}
        
    def CreatePlot(self, data, title, xlabel, ylabel, win, key):
            self.plots[win] = self.vis.line(data, opts=dict(
                title=title,
                xlabel=xlabel,
                ylabel=ylabel,
                win=win,
                name=key
            ))
    
    def UpdatePlot(self, x, y, win, key):
        self.vis.line(np.array([y]), X=np.array([x]), win=self.plots[win], name=key, update="append") 

    def Plot_Loss(self, key, epoch, loss):
        plot_win = "loss_window"
        if plot_win not in self.plots:
            self.CreatePlot(np.array([epoch, loss]), "Loss Graph", "Epoch", "Loss", plot_win, key)
        else:
            self.UpdatePlot(epoch, loss, plot_win, key)

vis = VisdomController()

Setting up a new session...


# Architechure

In [3]:
class FraudClassifier(torch.nn.Module):
    def __init__(self):
        super(FraudClassifier, self).__init__()
        self.hidden0 = nn.Sequential(
            nn.Linear(n_features, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        
        self.hidden1 = nn.Sequential(
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        
        self.hidden2 = nn.Sequential(
            nn.Linear(128, 64),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        
        self.hidden3 = nn.Sequential(
            nn.Linear(64, 32),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        
        self.hidden4 = nn.Sequential(
            nn.Linear(32, 16),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3)
        )
        
        self.out = nn.Sequential(
            nn.Linear(16, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x  = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.hidden3(x)
        x = self.hidden4(x)
        x = self.out(x)
        return x  
        

# Training Functions

In [33]:
num_epochs = 10
BCE = nn.BCELoss()
def check_accuracy(data, model):
    if data.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

def train_classifier_vanilla(network, optimizer, data):
    network = network.to(device=device)
    n = 0
    for epoch in range(num_epochs):
        epoch_loss = 0
        for n_batch, (x, y) in enumerate(data):
            n += 1
            network.train()
            x = x.to(device=device, dtype = dtype)
            y = y.to(device=device, dtype=dtype)
            t_start = millis = time.time()
            
            scores = network(x)
            loss = BCE(scores, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            #Loss History
            epoch_loss += loss
            
            if (n_batch % 100 == 0):
                display.clear_output(True)

                # Basic Data            
                print("Epoch {}, {} / {}".format(epoch, n_batch, len(data)))
                print("loss : ", loss)
            
                t_end = millis = time.time()
                print("Time Elapsed : ", t_end - t_start)
                print(n)
                vis.Plot_Loss("Vanilla Loss", n , loss.item())
            
        #vis.Plot_Loss("Vanilla Loss", epoch, epoch_loss.item() / len(data))
            

In [34]:
vanillaNN = FraudClassifier()
ADAMoptim = optim.Adam(vanillaNN.parameters(), lr = 1e-5)

train_classifier_vanilla(vanillaNN, ADAMoptim, data_loader)

Epoch 9, 2200 / 2279
loss :  tensor(0.0014, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)
Time Elapsed :  0.0040013790130615234
22712


In [61]:
vanillaNN.cuda()
x_test = x_test.cuda()
y_scores = vanillaNN(x_test)
average_precision = average_precision_score(y_test.detatch().cpu().numpy(), y_scores)
print('Average precision-recall score: ' + average_precision)

RuntimeError: CUDA error: an illegal memory access was encountered